<h1> DS200A Computer Vision Assignment</h1>

<h2>  Part One: Data Preparation </h2>	


<h3>  Import Statements </h3>	


In [42]:
#Import anything you need here
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib
import skimage
from skimage import data
from skimage import io
import os

from zipfile import ZipFile 
import re
from pathlib import Path

from skimage.color import rgb2gray, gray2rgb

In [43]:
#Example of reading in a file

# filename = os.path.join(skimage.data_dir, 'd.jpg')
# dog = io.imread(filename)
# skimage.io.imshow(dog)

# plt.show()
# filename2 = os.path.join(skimage.data_dir, 'c.jpg')
# cat = io.imread(filename2)
# example_data_frame= pd.DataFrame()


# example_data_frame['Pictures']=[dog, cat]
# example_data_frame['Encoding']=[1,2]
# example_data_frame

<h4> Using the code for reference, take the given folder and create a dataframe with the picture object, and the encoding as listed below. </h4>	
0=Airplanes, 1=Bear, 2=Blimp, 3=Comet, 4=Crab, 5=Dog, 6=Dolphin, 7=Giraffe, 8=Goat, 9=Gorilla, 10=Kangaroo, 11=Killer-Whale, 12=Leopards, 13=Llama, 14= Penguin, 15= Porcupine, 16=Teddy-Bear, 17=Triceratops, 18=Unicorn, 19=Zebra

In [44]:
def convert_grey(image):
    if len(image.shape) < 3:
        return gray2rgb(image)
    return image

In [45]:
def read_organize_data(isTest=False, cacheResults=True):
    #Fill this function out, should return a dataframe with picture object, and correct encoding
    
    # If cached, return file'
    if isTest:
        cacheName = 'cleaned_test.pkl'
    else:
        cacheName = 'cleaned_train.pkl'
        
    if Path(cacheName).is_file() and cacheResults:
        return pd.read_pickle(cacheName)
    
    image_names, folder_names, images = [], [], []
    
    # Define path
    if isTest:
        file_path = "20_Validation.zip"
    else:
        file_path = "20_categories_training.zip"
    
    # opening the zip file in READ mode 
    with ZipFile(file_path, 'r') as zip: 
        for info in zip.infolist():
            
            # Get file names
            img_name = re.findall("/(.*)", info.filename)            
            hidden_file = "._" in str(img_name) or " " in str(img_name)
            if img_name and not hidden_file:
                img_name = img_name[0]
                image_names.append(img_name) 
            
            # Get folder names and image
            folder_name = re.findall("(.*)/", info.filename)
            if folder_name and not hidden_file:
                folder_name = folder_name[0]
                folder_names.append(folder_name) 
                
                filename = "20_categories_training" / Path(folder_name) / Path(img_name)
                images.append(io.imread(filename))
    
    # Populate df
    df = pd.DataFrame()
    df['Folder name'] = folder_names
    df['Image name'] = image_names
    df['Image object'] = images
    
    # Convert grey-scale to RGB representative
    df['Image object'] = df['Image object'].apply(lambda x: convert_grey(x))
    
    encoding = {0:'airplanes', 1:'bear', 2:'blimp', 3:'comet', 4:'crab', \
                5:'dog', 6:'dolphin', 7:'giraffe', 8:'goat', 9:'gorilla', \
                10:'kangaroo', 11:'killer-whale', 12:'leopards', 13:'llama', \
                14:'penguin', 15:'porcupine', 16:'teddy-bear', 17:'triceratops', \
                18:'unicorn', 19:'zebra'}
    encoding = {v: k for k, v in encoding.items()}
    df['Encoding'] = df['Folder name'].map(encoding)
    
    # Cache
    if not Path(cacheName).is_file():
        df.to_pickle(f'./{cacheName}')
        
    return df

In [46]:
import time
t1 = time.time()
starting_data = read_organize_data(isTest=False, cacheResults=True)
t2=time.time()

print(t2-t1)

18.937308311462402


In [47]:
starting_data.head()

,Folder name,Image name,Image object,Encoding
0,airplanes,airplanes_0001.jpg,"[[[255, 255, 255], [255, 255, 255], [255, 255,...",0
1,airplanes,airplanes_0002.jpg,"[[[255, 255, 255], [255, 255, 255], [255, 255,...",0
2,airplanes,airplanes_0003.jpg,"[[[255, 255, 255], [255, 255, 255], [255, 255,...",0
3,airplanes,airplanes_0004.jpg,"[[[174, 190, 187], [173, 189, 186], [172, 188,...",0
4,airplanes,airplanes_0005.jpg,"[[[255, 255, 255], [255, 255, 255], [255, 255,...",0


### Testing

In [10]:
# testing changing grey to rgb values
image = np.array([[ [2, 0, 5],
                [4, 0, 5],
                [10, 1, 6],
                [12, 3, 7]],

               [[20, 4, 7],
                [40, 5, 8],
                [100, 5, 9],
                [120, 6, 10]]])
image = np.array([[2, 4, 10, 12, 3, 7], [20, 40, 100, 120, 150, 155]])

gray_to_rgb(image)

NameError: name 'gray_to_rgb' is not defined

In [ ]:
# testing imread and imshow

path = "20_categories_training\\bear"
filename = os.path.join(path, 'bear_0001.jpg')
print(filename)
dog = io.imread(filename)
skimage.io.imshow(dog)

In [ ]:
# testing regex
file = '_bear_0001.jpg'
test = re.findall(r'^[_.]', file)
if test:
    print("YES")
else:
    print("NO")